In [1]:
!pip install "chronos-forecasting[training] @ git+https://github.com/amazon-science/chronos-forecasting.git"

  Cloning https://github.com/amazon-science/chronos-forecasting.git to c:\users\rioti\appdata\local\temp\pip-install-69i8fau2\chronos-forecasting_4e62d8b0c5254a27a2329d369503e47f
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit 6a9c8dadac04eb85befc935043e3e2cce914267f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/527.3 kB ? eta -:--:--
   ---------------------------------------- 527.3/527.3 kB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
  

  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git 'C:\Users\rioti\AppData\Local\Temp\pip-install-69i8fau2\chronos-forecasting_4e62d8b0c5254a27a2329d369503e47f'

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Another possible way to install chronos for fine-tuning

# Install in editable mode with extra training-related dependencies
!cd chronos-forecasting && pip install --editable ".[training]"


Obtaining file:///C:/Users/rioti/Documents/GitHub/NLP_prediction-and-forecasting/chronos-forecasting
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
  Building editable for chronos-forecasting (pyproject.toml): started
  Building editable for chronos-forecasting (pyproject.toml): finished with status 'done'
  Create


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from pathlib import Path
from typing import List, Union
import pandas as pd
import numpy as np
from gluonts.dataset.arrow import ArrowWriter


def convert_to_arrow(
    path: Union[str, Path],
    time_series: Union[List[np.ndarray], np.ndarray],
    compression: str = "lz4",
):
    """
    Store a given set of series into Arrow format at the specified path.

    Input data can be either a list of 1D numpy arrays, or a single 2D
    numpy array of shape (num_series, time_length).
    """
    assert isinstance(time_series, list) or (
        isinstance(time_series, np.ndarray) and
        time_series.ndim == 2
    )

    # Set an arbitrary start time
    start = np.datetime64("2000-01-01 00:00", "s")

    dataset = [
        {"start": start, "target": ts} for ts in time_series
    ]

    ArrowWriter(compression=compression).write_to_file(
        dataset,
        path=path,
    )


if __name__ == "__main__":
    csv_dir = Path("training_data")
    arrow_dir = Path("training_data_arrow")
    arrow_dir.mkdir(parents=True, exist_ok=True)

    # cicla su tutti i file .csv nella directory
    for csv_path in csv_dir.glob("*.csv"):
        # leggi il CSV
        df = pd.read_csv(csv_path)
        # estrai la serie (1D numpy array)
        ts = df["sea_level_altitude"].to_numpy(dtype=float)
        # costruisci il nome del file .arrow
        arrow_path = arrow_dir / f"{csv_path.stem}.arrow"
        # convertilo
        convert_to_arrow(
            path=str(arrow_path),
            time_series=[ts],
            compression="lz4"   # o un’altra compressione che preferisci
        )
        print(f"→ creato: {arrow_path}")

→ creato: training_data_arrow\ADRIATIC_SEA.arrow
→ creato: training_data_arrow\ADRIATIC_SEA_no_season.arrow
→ creato: training_data_arrow\ADRIATIC_SEA_no_season_outside_66.arrow
→ creato: training_data_arrow\ADRIATIC_SEA_outside_66.arrow
→ creato: training_data_arrow\ANDAMAN_SEA.arrow
→ creato: training_data_arrow\ANDAMAN_SEA_no_season.arrow
→ creato: training_data_arrow\ANDAMAN_SEA_no_season_outside_66.arrow
→ creato: training_data_arrow\ANDAMAN_SEA_outside_66.arrow
→ creato: training_data_arrow\ARABIAN_SEA.arrow
→ creato: training_data_arrow\ARABIAN_SEA_no_season.arrow
→ creato: training_data_arrow\ARABIAN_SEA_no_season_outside_66.arrow
→ creato: training_data_arrow\ARABIAN_SEA_outside_66.arrow
→ creato: training_data_arrow\ATLANTIC_OCEAN.arrow
→ creato: training_data_arrow\ATLANTIC_OCEAN_no_season.arrow
→ creato: training_data_arrow\ATLANTIC_OCEAN_no_season_outside_66.arrow
→ creato: training_data_arrow\ATLANTIC_OCEAN_outside_66.arrow
→ creato: training_data_arrow\BALTIC_SEA.arrow
→

In [2]:
from pathlib import Path
from ruamel.yaml import YAML

# Inizializza il parser YAML
yaml = YAML()

# Percorso del file di config originale
cfg_path = Path(r"C:/Users/rioti/Documents/GitHub/NLP_prediction-and-forecasting/chronos-forecasting/scripts/training/configs/chronos-t5-tiny-modified.yaml")

# Carica il config esistente
with cfg_path.open("r") as f:
    cfg = yaml.load(f)

# Directory dove risiedono gli .arrow
arrow_dir = Path(r"C:/Users/rioti/Documents/GitHub/NLP_prediction-and-forecasting/training_data_arrow")

# Trova e ordina tutti i file .arrow
files = sorted(arrow_dir.glob("*.arrow"))

# Ricostruisci le liste training_data_paths e probability
cfg["training_data_paths"] = [str(p) for p in files]
n = len(files)
cfg["probability"] = [1.0 / n] * n

# Salva in un nuovo file (nella stessa cartella del config originale)
output_path = cfg_path.parent / "chronos-t5-tiny-modified_auto.yaml"
with output_path.open("w") as f:
    yaml.dump(cfg, f)

print(f"Generati {n} path in {output_path}")


Generati 88 path in C:\Users\rioti\Documents\GitHub\NLP_prediction-and-forecasting\chronos-forecasting\scripts\training\configs\chronos-t5-tiny-modified_auto.yaml


In [3]:
import subprocess
proc = subprocess.Popen(
    ["python", "-u", "chronos-forecasting/scripts/training/train.py",
     "--config", "chronos-forecasting/scripts/training/configs/chronos-t5-tiny-modified_auto.yaml"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,                    # alias di universal_newlines=True
    encoding="utf-8",             # forza utf-8
    errors="replace"              # o "ignore" se preferisci silenziare i caratteri non decodificabili
)
for line in proc.stdout:
    print(line, end="")


2025-07-11 15:51:28,304 - c:\Users\rioti\Documents\GitHub\NLP_prediction-and-forecasting\chronos-forecasting\scripts\training\train.py - INFO - Using SEED: 457093429
2025-07-11 15:51:28,429 - c:\Users\rioti\Documents\GitHub\NLP_prediction-and-forecasting\chronos-forecasting\scripts\training\train.py - INFO - Logging dir: output\run-25
2025-07-11 15:51:28,429 - c:\Users\rioti\Documents\GitHub\NLP_prediction-and-forecasting\chronos-forecasting\scripts\training\train.py - INFO - Loading and filtering 88 datasets for training: ['C:\\Users\\rioti\\Documents\\GitHub\\NLP_prediction-and-forecasting\\training_data_arrow\\ADRIATIC_SEA.arrow', 'C:\\Users\\rioti\\Documents\\GitHub\\NLP_prediction-and-forecasting\\training_data_arrow\\ADRIATIC_SEA_no_season.arrow', 'C:\\Users\\rioti\\Documents\\GitHub\\NLP_prediction-and-forecasting\\training_data_arrow\\ADRIATIC_SEA_no_season_outside_66.arrow', 'C:\\Users\\rioti\\Documents\\GitHub\\NLP_prediction-and-forecasting\\training_data_arrow\\ADRIATIC_SEA